In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.show()

Py4JJavaError: An error occurred while calling o35.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (Gamer executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1105)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1091)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:571)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$4(Executor.scala:935)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$4$adapted(Executor.scala:932)
	at scala.collection.TraversableLike$WithFilter.$anonfun$fore

In [ ]:
# Filter by voters.
df2 = video_games_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show(10)

df3 = df2.filter(df2['total_votes'] >= 20)
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)

In [ ]:
# Describe stats.
%pyspark
from pyspark.sql.functions import col, avg
paid_df = df4.filter(df4['vine']== 'Y')
unpaid_df = df4.filter(df4['vine']== 'N')

paid_df.describe().show()
unpaid_df.describe().show()

In [9]:
#  Determine the percentage of five-star reviews among Vine reviews
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

NameError: name 'paid_df' is not defined

In [10]:
#  Determine the percentage of five-star reviews among non-Vine reviews.
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)
print(unpaid_number)
print(unpaid_five_star_number)
print(float(unpaid_five_star_number) / float(unpaid_number))

NameError: name 'unpaid_df' is not defined